#### Exercise one:
Revisit the [sample code here](W1_4_Data_Structures.ipynb) from week 1 which contains data in a number of different data structures, such as list[] and dict(). Using these structures within a program and see what you can convert into either a pandas's Series or DataFrame. 

#### Exercise two:
Import the [following .csv file](SalesData.csv) into an appropriate data structure in pandas. Using the functions available in pandas and NumPy (where they are more appropriate) output answers to the following questions:

- What were the sales for P2 and B8 in Nov-18, Feb-18 and Mar-19?
- In the third quarter (Oct-18 to Dec-18) what were the sales figures for London (L3 & 1) and what was the monthly percentage increase?
- What were the top three months for New York (N6 and N4) and in which stores did they occur?
- What was the overall lowest sales figure in which store and which month?

In [76]:
import pandas as pd
import numpy as np
gap ="\n--------------------------------------------------------------------\n"
df = pd.read_csv("SalesData.csv")
df

,Unnamed: 0,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18,Nov-18,Dec-18,Jan-19,Feb-19,Mar-19
0,L3,390,345,379,353,365,326,363,459,539,474,340,343
1,L1,392,377,347,395,330,332,340,446,573,464,223,216
2,P2,250,270,300,398,376,382,306,489,552,488,298,269
3,N6,460,480,450,310,392,383,314,402,640,495,215,243
4,N4,460,480,450,387,388,320,337,478,676,408,338,263
5,B8,470,510,360,472,423,385,353,464,598,419,459,659


In [77]:
#What were the sales for P2 and B8 in Nov-18, Feb-19 and Mar-19?

p2_nov_feb_Mar = df.iloc[3,[8,11,12]] #using indexes
b8_nov_feb_Mar = df.iloc[5,[8,11,12]] #using indexes
print(f'P2 Sales in Nov, Feb and March are:\n{p2_nov_feb_Mar}')
print(gap)
print(f'B8 Sales in Nov, Feb and March are:\n{b8_nov_feb_Mar}')



P2 Sales in Nov, Feb and March are:
Nov-18    402
Feb-19    215
Mar-19    243
Name: 3, dtype: object

--------------------------------------------------------------------

B8 Sales in Nov, Feb and March are:
Nov-18    464
Feb-19    459
Mar-19    659
Name: 5, dtype: object


In [100]:
#In the third quarter (Oct-18 to Dec-18) 
# what were the sales figures for London (L3 & 1) 
# and what was the monthly percentage increase?

l3_l2_3rd_quarter_figures = df.iloc[:2,[7,8,9]] #using indexes
print(f'3rd Quarter Sales for  London:\n{l3_l2_3rd_quarter_figures}')
print(gap)

#calculate monthly percentage increase using pct_change()
monthly_percentage_increase = l3_l2_3rd_quarter_figures.pct_change()
print(f'3rd Quarter Sales percentage Increase:\n{monthly_percentage_increase *100}%')

3rd Quarter Sales for  London:
   Oct-18  Nov-18  Dec-18
0     363     459     539
1     340     446     573

--------------------------------------------------------------------

3rd Quarter Sales percentage Increase:
     Oct-18    Nov-18    Dec-18
0       NaN       NaN       NaN
1 -6.336088 -2.832244  6.307978%


In [103]:
#What were the top three months for New York (N6 and N4) 
# and in which stores did they occur?

my_stores = df[df['Unnamed: 0'].isin(['N6','N4'])]
print(my_stores)

#find top 3 months for NewYork Stores
top_three_months = my_stores.drop(columns='Unnamed: 0').stack().nlargest(3)
print(gap)
print(f'Top three months for New York stores: \n{top_three_months}')

#Extract stores where top 3 month occured using boolean indexing
stores_with_top_three_months = my_stores.set_index('Unnamed: 0').loc[:, top_three_months.index.get_level_values(1)].dropna()


print(gap)
print(f'Stores where the top 3 months occurred: \n{stores_with_top_three_months}')

  Unnamed: 0  Apr-18  May-18  Jun-18  Jul-18  Aug-18  Sep-18  Oct-18  Nov-18  \
3         N6     460     480     450     310     392     383     314     402   
4         N4     460     480     450     387     388     320     337     478   

   Dec-18  Jan-19  Feb-19  Mar-19  
3     640     495     215     243  
4     676     408     338     263  

--------------------------------------------------------------------

Top three months for New York stores: 
4  Dec-18    676
3  Dec-18    640
   Jan-19    495
dtype: int64

--------------------------------------------------------------------

Stores where the top 3 months occurred: 
            Dec-18  Dec-18  Jan-19
Unnamed: 0                        
N6             640     640     495
N4             676     676     408


In [104]:
# What was the overall lowest sales figure in which store and which month?

# Find the overall lowest sales figure and its index (store and month)
min_sales_index = df.drop(columns='Unnamed: 0').values.argmin()
min_store, min_month = divmod(min_sales_index, len(df.columns) - 1)
min_store_name = df.loc[min_store, 'Unnamed: 0']
min_month_name = df.columns[min_month + 1]

print(f"The overall lowest sales figure was {df.iloc[min_store, min_month + 1]} "
      f"in the store '{min_store_name}' during the month of '{min_month_name}'.")

The overall lowest sales figure was 215 in the store 'N6' during the month of 'Feb-19'.


### Exercise three:
Using the data set from exercise 2, above, create a new data table, including headings, that gives the percentage increase for each month, based on the stores average for the year. From this ascertain:

Which store had the largest and smallest increase and in which months did they occur?
Across all the stores which months showed the smallest average increase and the largest average increase?

In [ ]:
# Set 'Unnamed: 0' as the index for the DataFrame
df.set_index('Unnamed: 0', inplace=True)

# Calculate the average sales for each store
store_averages = df.drop(columns='Unnamed: 0').mean(axis=1)

# Calculate the percentage increase for each month relative to the store's average
percentage_increase_df = (df.drop(columns='Unnamed: 0') - store_averages.values.reshape(-1,1)) / store_averages.values.reshape(-1,1) * 100

# Create a new DataFrame with headings (store names and months)
percentage_increase_table = pd.DataFrame(data=percentage_increase_df.values, index=df['Unnamed: 0'], columns=df.columns[1:])

# Find the store with the largest and smallest increase and in which months they occurred
largest_increase_store = percentage_increase_table.idxmax().loc['Unnamed: 0']
largest_increase_month = percentage_increase_table.loc[largest_increase_store].idxmax()
smallest_increase_store = percentage_increase_table.idxmin().loc['Unnamed: 0']
smallest_increase_month = percentage_increase_table.loc[smallest_increase_store].idxmin()

# Find the months with the smallest and largest average increase across all stores
avg_increase_per_month = percentage_increase_table.mean()
smallest_avg_increase_month = avg_increase_per_month.idxmin()
largest_avg_increase_month = avg_increase_per_month.idxmax()

print("Percentage Increase Table:")
print(percentage_increase_table)

print("\nStore with the largest increase:", largest_increase_store, "in the month of", largest_increase_month)
print("Store with the smallest increase:", smallest_increase_store, "in the month of", smallest_increase_month)

print("\nMonth with the smallest average increase across all stores:", smallest_avg_increase_month)
print("Month with the largest average increase across all stores:", largest_avg_increase_month)
